## Atividades Desenvolvidas no Notebook - 05_carga_dados_sql_dw

|Sequência|Ação|Detalhamento
|---|---|---|
|SEQ-01 |Função acentuacao|Usada para remover a acentuação de campos do tipo STRING.|
|SEQ-02 / SEQ-03 / SEQ-04 / SEQ-05 / SEQ-06|Ingestão dos Dados da Camada Bronze para Silver|Carga realizada baseada nos dados existentes nas tabelas da camada Bronze.|
|SEQ-02 / SEQ-03 / SEQ-04 / SEQ-05 / SEQ-06|Tratamento de dados antes de inserir/atualizar|Remover acentuação e caracteres especiais, valores nulos onde o datatype foi STRING, informar 'N/A', datatype NUMÉRICO, informar 0 (zero) e datatype DATE/TIMESTAMP, informar '1900-01-01'. Todo os dados do tipo STRING devem estar dados em caixa alta.|
|SEQ-02 / SEQ-03 / SEQ-04 / SEQ-05 / SEQ-06|Identificar a data de alteração ou cadastro|Regra: Caso o registro não exista (caindo na condição INSERT do MERGE), a data de cadastro e data de alteração devem ser a mesma, caso o registro exista, a data de alteração deverá ser alterada.|

In [0]:
from pyspark.sql.functions import *
from datetime import date, timedelta
import pyodbc

In [0]:
pip install pyodbc

In [0]:
dfjson =  pd.read_json("https://raw.githubusercontent.com/dbaassists/Projeto_BI_Zero_TO_DW/main/04_ARQUIVO_CONFIG/config_azure_sql.json")

server = dfjson['Config']['server']
database = dfjson['Config']['database']
username = dfjson['Config']['username']
password = dfjson['Config']['password']

print(server)

srv-db-treinamentosql.database.windows.net:1433


In [0]:
%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   983  100   983    0     0   7503      0 --:--:-- --:--:-- --:--:--  7503
OK
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    79  100    79    0     0    681      0 --:--:-- --:--:-- --:--:--   675
Hit:1 https://packages.microsoft.com/ubuntu/16.04/prod xenial InRelease
Hit:2 https://repos.azul.com/zulu/deb stable InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubunt

In [0]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=srv-db-treinamentosql.database.windows.net;port=1433;DATABASE=srv-db-treinamentosql;UID='+username+';PWD='+ password)

lista_tabela = ['dw.Fato_Venda', 'dw.Dim_Cliente', 'dw.Dim_Produto', 'dw.Dim_Forma_Pagamento', 'dw.Dim_Tempo', 'dw.Dim_Loja']

for tabela in lista_tabela:

    print(tabela)

    cursor = cnxn.cursor()

    if tabela == 'dw.Fato_Venda':
        comando = 'TRUNCATE TABLE'
        print(comando)
        cursor.execute(f'TRUNCATE TABLE {tabela}')

    else:

        comando = 'DELETE FROM'
        print(comando)
        cursor.execute(f'DELETE FROM {tabela}')    

    cursor.commit()

cnxn.close()    

dw.Fato_Venda
TRUNCATE TABLE
dw.Dim_Cliente
DELETE FROM
dw.Dim_Produto
DELETE FROM
dw.Dim_Forma_Pagamento
DELETE FROM
dw.Dim_Tempo
DELETE FROM
dw.Dim_Loja
DELETE FROM


In [0]:

# Consulta SQL para selecionar os dados do Databricks SQL
query = """
SELECT * FROM gold.dim_cliente
"""
# Carregando os dados do Databricks SQL
df = spark.sql(query)

# Escrevendo os dados no banco de dados Azure SQL
df.write \
    .format("jdbc") \
    .option("url",  f"jdbc:sqlserver://{server};database={database}") \
    .option("dbtable", "dw.Dim_Cliente") \
    .option("user", username) \
    .option("password", password) \
    .mode("append") \
    .save()

In [0]:

# Consulta SQL para selecionar os dados do Databricks SQL
query = """
SELECT * FROM gold.dim_produto
"""
# Carregando os dados do Databricks SQL
df = spark.sql(query)

# Escrevendo os dados no banco de dados Azure SQL
df.write \
    .format("jdbc") \
    .option("url",  f"jdbc:sqlserver://{server};database={database}") \
    .option("dbtable", "dw.Dim_Produto") \
    .option("user", username) \
    .option("password", password) \
    .mode("append") \
    .save()

In [0]:

# Consulta SQL para selecionar os dados do Databricks SQL
query = """
SELECT * FROM gold.dim_forma_pagamento
"""
# Carregando os dados do Databricks SQL
df = spark.sql(query)

# Escrevendo os dados no banco de dados Azure SQL
df.write \
    .format("jdbc") \
    .option("url",  f"jdbc:sqlserver://{server};database={database}") \
    .option("dbtable", "dw.Dim_Forma_Pagamento") \
    .option("user", username) \
    .option("password", password) \
    .mode("append") \
    .save()

In [0]:

# Consulta SQL para selecionar os dados do Databricks SQL
query = """
SELECT * FROM gold.dim_tempo
"""
# Carregando os dados do Databricks SQL
df = spark.sql(query)

# Escrevendo os dados no banco de dados Azure SQL
df.write \
    .format("jdbc") \
    .option("url",  f"jdbc:sqlserver://{server};database={database}") \
    .option("dbtable", "dw.Dim_Tempo") \
    .option("user", username) \
    .option("password", password) \
    .mode("append") \
    .save()

In [0]:

# Consulta SQL para selecionar os dados do Databricks SQL
query = """
SELECT * FROM gold.dim_loja
"""
# Carregando os dados do Databricks SQL
df = spark.sql(query)

# Escrevendo os dados no banco de dados Azure SQL
df.write \
    .format("jdbc") \
    .option("url",  f"jdbc:sqlserver://{server};database={database}") \
    .option("dbtable", "dw.Dim_Loja") \
    .option("user", username) \
    .option("password", password) \
    .mode("append") \
    .save()

In [0]:

# Consulta SQL para selecionar os dados do Databricks SQL
query = """
SELECT * FROM gold.fato_venda
"""
# Carregando os dados do Databricks SQL
df = spark.sql(query)

# Escrevendo os dados no banco de dados Azure SQL
df.write \
    .format("jdbc") \
    .option("url",  f"jdbc:sqlserver://{server};database={database}") \
    .option("dbtable", "dw.Fato_Venda") \
    .option("user", username) \
    .option("password", password) \
    .mode("append") \
    .save()